In [53]:
from pprint import pprint
import random

In [54]:
def create_user_interest_dict(*interests):
    interest_count = len(interests)
    interest_weight = round((100-10) / interest_count,2) # Subtract 10 for 'Random'
    user_interests = {interest: interest_weight for interest in interests}
    user_interests['Random'] = 10 # Add 'Random' with constant value of 10
    return user_interests

In [55]:
# HELPER FUNCTIONS for update_weight

def observe_action_taken(video_tag, liked, shared, watched, loop_count):
    """
    This function calculates the total points based on the user's actions.

    Parameters:
    video_tag (str): The tag of the video.
    liked (bool): Whether the user liked the video.
    shared (bool): Whether the user shared the video.
    watched (bool): Whether the user watched the video.
    loop_count (int): The number of times the user looped the video.

    Returns:
    float: The total points calculated based on the user's actions.
    """
    # Actions and their corresponding points to add
    Actions = {
        "Like": 0.1,
        "Share": 0.2,
        "Watch": 0.1,  # more than 50% of total duration
        # todo: experiment with increasing watch value
        "Loop": 0.05,
    }

    total_points = 0

    if liked:
        total_points += Actions["Like"]
    if shared:
        total_points += Actions["Share"]
    if watched:
        total_points += Actions["Watch"]
    total_points += loop_count * Actions["Loop"]

    print(f"\nUpdating weights for video tag: {video_tag}")
    print(f"Total points to add: {total_points}")

    return total_points

def handle_update_weight(user_dict_interest, video_tag, total_points):
    """
    This function updates the user's interest score based on the total points.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.
    total_points (float): The total points calculated based on the user's actions.

    Returns:
    dict: The updated user's interest dictionary.
    """
    # Increase the weight of the interest based on the action points
    user_dict_interest[video_tag] += total_points
    print(f"Updated {video_tag} to {user_dict_interest[video_tag]}\n")

    # Deduct the total points proportionally from existing interests (excluding 'Random' and the video_tag itself)
    total_deductable_weight = sum(
        weight for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag

    )
    print(f"Total weight to deduct from other interests: {total_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}")

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (total_points / total_deductable_weight) * user_dict_interest[interest]
            user_dict_interest[interest] -= deduction

            print(f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}")

    return user_dict_interest

def handle_add_new_tag(user_dict_interest, video_tag):
    """
    This function adds a new interest to the user's interest dictionary.

    Parameters:
    user_dict_interest (dict): The user's interest dictionary.
    video_tag (str): The tag of the video.

    Returns:
    dict: The updated user's interest dictionary with the new interest added.
    """
    # Add new interest if it doesn't exist and give initial points
    initial_points = 5
    user_dict_interest[video_tag] = initial_points
    print(f"Added new interest {video_tag} with initial weight: {initial_points}")

    # Deduct initial points proportionally from existing interests (excluding 'Random' and current video_tag)
    total_deductable_weight = sum(
        weight
        for interest, weight in user_dict_interest.items()
        if interest != "Random" and interest != video_tag
    )

    print(f"Total weight to deduct from other interests: {initial_points}\nTotal deductable weight (excluding 'Random' and {video_tag}): {total_deductable_weight}\n")

    for interest in user_dict_interest:
        if interest != "Random" and interest != video_tag:
            deduction = (
                initial_points / total_deductable_weight
            ) * user_dict_interest[interest]
            user_dict_interest[interest] -= deduction

            pprint(f"Deducted {deduction} from {interest}, new weight: {user_dict_interest[interest]}")

    return user_dict_interest

In [56]:
def watch_video_and_update_weight(
    user_dict_interest,
    video_tag,
    liked=False,
    shared=False,
    watched=False,
    loop_count=0,
):

    total_points = observe_action_taken(video_tag, liked, shared, watched, loop_count)

    if video_tag in user_dict_interest:
        handle_update_weight(user_dict_interest, video_tag, total_points)
    else:
        handle_add_new_tag(user_dict_interest, video_tag)

    # Sort the interests ("key=item[1]") by weight in descending order and keep only the top 10
    user_dict_interest = dict(
        sorted(user_dict_interest.items(), key=lambda item: item[1], reverse=True)[:10]
    )

    # Check total weight to ensure it sums to 100 (for debugging purposes)
    total_weight = sum(user_dict_interest.values())
    print(f"Total weight after update: {total_weight} (should be 100)\n")
    print(f"Updated interests: {user_dict_interest}\n")

    return user_dict_interest

In [57]:
def get_random_tag_for_recommendation(user_dict_interest):

    random_value = round(random.uniform(0, 100), 0)

    print("_"*50)
    print(f"\nRandom value: {random_value}\n")

    cumulative_probability = 0.0
    partitions = []

    for interest, probability in user_dict_interest.items():
        # Save the starting point of the current tag's range (in probability)
        previous_cumulative_probability = cumulative_probability

        # Add the tag's probability to the cumulative total to get the end point of the tag's range
        cumulative_probability += probability

        partitions.append(
            (interest, previous_cumulative_probability, cumulative_probability)
        )

        pprint(
            f"Checking interest: {interest}, cumulative range: {previous_cumulative_probability} - {cumulative_probability}"
        )

        # If the random value is less than or equal to the cumulative probability up to the current interest,
        # select the current interest and stop looking at the rest of the interests.
        if random_value <= cumulative_probability:
            print(f"\nSelected interest: {interest}\n")
            break

    return interest

In [58]:
# New user selects interests (max 5)
# 10 total - 5 start to discover other 5 naturally

carl_interests = create_user_interest_dict(
    "DevOps",
    "Tests (SAT)",
    "Machine Learning",
    "Cats",
    "Cooking",
    "Edgur",
    "Gardening",
    "Music",
    "Dogs",
    "Travel",
    "Eleven",
)
pprint(carl_interests)

{'Cats': 8.18,
 'Cooking': 8.18,
 'DevOps': 8.18,
 'Dogs': 8.18,
 'Edgur': 8.18,
 'Eleven': 8.18,
 'Gardening': 8.18,
 'Machine Learning': 8.18,
 'Music': 8.18,
 'Random': 10,
 'Tests (SAT)': 8.18,
 'Travel': 8.18}


In [63]:
# Simulate user actions
# pprint(f"Before update: {carl_interests}")

watch_video_and_update_weight(carl_interests, "NewTag", liked=True, watched=True, shared=True, loop_count=10)

# pprint(f"After update: {carl_interests}")


Updating weights for video tag: NewTag
Total points to add: 0.9
Updated NewTag to 7.700000000000001

Total weight to deduct from other interests: 0.9
Total deductable weight (excluding 'Random' and NewTag): 83.17999999999999
Deducted 0.08181818181818183 from DevOps, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Tests (SAT), new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Machine Learning, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Cats, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Cooking, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Edgur, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Gardening, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Music, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Dogs, new weight: 7.4799999999999995
Deducted 0.08181818181818183 from Travel, new weight: 7.4799999999999995
Deducted 0.08181818181818183 fr

{'Random': 10,
 'NewTag': 7.700000000000001,
 'DevOps': 7.4799999999999995,
 'Tests (SAT)': 7.4799999999999995,
 'Machine Learning': 7.4799999999999995,
 'Cats': 7.4799999999999995,
 'Cooking': 7.4799999999999995,
 'Edgur': 7.4799999999999995,
 'Gardening': 7.4799999999999995,
 'Music': 7.4799999999999995}

In [60]:
# Simulate algorithm picking the next video to recommend via tag
selected_interest = get_random_tag_for_recommendation(carl_interests)

__________________________________________________

Random value: 61.0

'Checking interest: DevOps, cumulative range: 0.0 - 7.725454545454546'
('Checking interest: Tests (SAT), cumulative range: 7.725454545454546 - '
 '15.450909090909091')
('Checking interest: Machine Learning, cumulative range: 15.450909090909091 - '
 '23.176363636363636')
('Checking interest: Cats, cumulative range: 23.176363636363636 - '
 '30.901818181818182')
('Checking interest: Cooking, cumulative range: 30.901818181818182 - '
 '38.627272727272725')
('Checking interest: Edgur, cumulative range: 38.627272727272725 - '
 '46.35272727272727')
('Checking interest: Gardening, cumulative range: 46.35272727272727 - '
 '54.07818181818182')
('Checking interest: Music, cumulative range: 54.07818181818182 - '
 '61.803636363636365')

Selected interest: Music



In [ ]:
# handle random
# if selected_interest == "Random":
#     print("Random video selected, picking a random interest")
#     selected_interest = get_random_interest(UNIVERSE OF ALL INTERESTS IN CSV)

In [ ]:
# Serve video based on selected interest

# get videos from
    # https://docs.google.com/spreadsheets/d/e/2PACX-1vSAE2tBAnAdXsxk9a9YClFN7MSEVhzEmJD01ewwtooMLxL-Ilod26EbdD8sZeZk0ybiqD-jqT-9RZbn/pub?gid=497214901&single=true&output=csv

# chromaDB query("selected_interest")
    # get vid with closest distance

In [ ]:
# remove video from queue (or mark as watched)
# ID not title to handle duplicates


In [ ]:
# random.choice tag from selected video
    # run weights thing all over again